In [4]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
xls = pd.ExcelFile(r"C:\Users\Akshaj Kabthiyal\Desktop\KPMG Intern\1 Data Quality Assessment\KPMG_VI_New_raw_data_update_final.xlsx")
pd3 = pd.read_excel(xls, sheet_name=3, header=1)

C:\Users\Akshaj Kabthiyal\AppData\Local\Temp\ipykernel_5036\4213319220.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  pd3 = pd.read_excel(xls, sheet_name=3, header=1)


In [6]:
df3 = pd.DataFrame(pd3)

In [7]:
# check out datatypes, columns name, counts
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            3875 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3494 non-null   object        
 7   job_industry_category                3344 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  default     

In [8]:
# check the first 5 records
df3.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [9]:
# check for null values
df3.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

In [10]:
# drop the column if too many datapoint is missing to be usable
drop_threshold = df3.shape[0]*0.5
df3 = df3.dropna(thresh=drop_threshold, how='all', axis='columns').copy()

In [13]:
# sanity check for duplicate data
column_names = df3.columns

for i in column_names:
    print((i, df3[i].is_unique))

('customer_id', True)
('first_name', False)
('last_name', False)
('gender', False)
('past_3_years_bike_related_purchases', False)
('DOB', False)
('job_title', False)
('job_industry_category', False)
('wealth_segment', False)
('deceased_indicator', False)
('default', False)
('owns_car', False)
('tenure', False)


In [14]:
# drop the column with corrupted data
del df3['default']

In [15]:
# make sure the ID only contain legitimate digits
df3['customer_id'] = df3['customer_id'].astype('int64')

In [16]:
# make sure that gender is a category
df3['gender'] = df3['gender'].astype('category')

In [17]:
# check the entered data
df3['gender'].value_counts()

Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: gender, dtype: int64

In [18]:
# replace all the misspelt or differently formatted data
df3['gender'].replace({'M' : 'Male', 'F' : 'Female', 'Femal' : 'Female'}, inplace=True)

In [19]:
# check the frequency of different job titles
df3['job_title'].value_counts()

Business Systems Development Analyst    45
Tax Accountant                          44
Social Worker                           44
Internal Auditor                        42
Recruiting Manager                      41
                                        ..
Database Administrator I                 4
Health Coach I                           3
Health Coach III                         3
Research Assistant III                   3
Developer I                              1
Name: job_title, Length: 195, dtype: int64

In [20]:
# fill the missing datapoint using the backward filling method
df3['job_title']= df3['job_title'].fillna(method='bfill')

In [21]:
# check the frequency of job industries
df3['job_industry_category'].value_counts()

Manufacturing         799
Financial Services    774
Health                602
Retail                358
Property              267
IT                    223
Entertainment         136
Argiculture           113
Telecommunications     72
Name: job_industry_category, dtype: int64

In [22]:
# fill the missing datapoint using the forward filling method
df3['job_industry_category']= df3['job_industry_category'].fillna(method='ffill')

In [24]:
# make sure all dates are in the correct format
df3['DOB'] = df3['DOB'].dt.date

In [25]:
# convert to boolean data type
f = {'N': False, 'Y': True}
df3['deceased_indicator'] = df3['deceased_indicator'].map(f).fillna(df3['deceased_indicator'])

In [26]:
# convert to boolean data type
g = {'No': False, 'Yes': True}
df3['owns_car'] = df3['owns_car'].map(g).fillna(df3['owns_car'])

In [27]:
# calculate the median value for tenure
np.nanmedian(df3['tenure'])

11.0

In [28]:
# filling in the missing value for tenure
df3['tenure'].fillna(value = 11.0, inplace = True)

In [30]:
# make sure that the column names are in the correct format
df3.columns = map(str.lower, df3.columns)
df3.columns = map(str.strip, df3.columns)

In [32]:
#check the dataset after cleaning
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   customer_id                          4000 non-null   int64   
 1   first_name                           4000 non-null   object  
 2   last_name                            3875 non-null   object  
 3   gender                               4000 non-null   category
 4   past_3_years_bike_related_purchases  4000 non-null   int64   
 5   dob                                  3913 non-null   object  
 6   job_title                            4000 non-null   object  
 7   job_industry_category                4000 non-null   object  
 8   wealth_segment                       4000 non-null   object  
 9   deceased_indicator                   4000 non-null   bool    
 10  owns_car                             4000 non-null   bool    
 11  tenure           

In [34]:
df3

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,dob,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,False,True,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,False,True,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,False,True,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,Senior Editor,IT,Mass Customer,False,False,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,IT,Affluent Customer,False,True,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,False,False,19.0
3996,3997,Blanch,Nisuis,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,False,True,1.0
3997,3998,Sarene,Woolley,U,60,NaT,Assistant Manager,IT,High Net Worth,False,False,11.0
3998,3999,Patrizius,NaN,Male,11,1973-10-24,Software Engineer IV,Manufacturing,Affluent Customer,False,True,10.0


In [35]:
#print to csv file after the cleaning
df3.to_csv('./data3_cleaned.csv',index=False)